# Benchmark filtering thresholds

Benchmark **in_soma_thr** (first entry only) and **nc_thr** on detection outcomes using pre-computed `all_granules.parquet`. Filtering is applied on the granule dataframe only (no detection rerun). Preprocessing: size filter `sphere_r < 4`; NC filtering uses all `nc_genes` and transcript kdtree (no top-15 selection). Threshold = 1 turns off that filter.

In [3]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scanpy as sc
from mcDETECT.model import mcDETECT
from scipy.spatial import cKDTree

import warnings
warnings.filterwarnings("ignore")

In [4]:
dataset = "MERSCOPE_WT_1"
data_path = f"../../data/{dataset}/"
output_path = f"../../output/{dataset}/"
benchmark_path = "../../output/benchmark/"

In [5]:
granules = pd.read_parquet(output_path + "all_granules_local.parquet")
granules = granules[granules["sphere_r"] < 4].copy().reset_index(drop=True)
print(f"Granules after size filter (sphere_r < 4): {len(granules)}")

Granules after size filter (sphere_r < 4): 1148130


In [6]:
transcripts = pd.read_parquet(data_path + "processed_data/transcripts.parquet")
nc_genes = list(pd.read_csv(data_path + "processed_data/negative_controls.csv")["Gene"])

In [9]:
def make_tree_3d(d1, d2, d3):
    """Build 3D cKDTree from coordinate arrays."""
    points = np.c_[np.ravel(d1), np.ravel(d2), np.ravel(d3)]
    return cKDTree(points)


def compute_nc_ratio(granules, transcripts, nc_genes):
    """Per granule: nc_ratio = (NC transcript count in sphere) / size. Center (sphere_x,y,z), radius sphere_r."""
    nc_trans = transcripts[transcripts["target"].isin(nc_genes)]
    if nc_trans.shape[0] == 0:
        return np.zeros(len(granules))
    tree = make_tree_3d(
        nc_trans["global_x"].to_numpy(),
        nc_trans["global_y"].to_numpy(),
        nc_trans["global_z"].to_numpy(),
    )
    centers = granules[["sphere_x", "sphere_y", "layer_z"]].to_numpy()
    radii = granules["sphere_r"].to_numpy()
    sizes = granules["size"].to_numpy().astype(float)
    counts = np.array([len(tree.query_ball_point(c, r)) for c, r in zip(centers, radii)])
    return np.where(sizes > 0, counts / sizes, 0.0)


def apply_filters(granules, nc_ratio, in_soma_thr, nc_thr):
    """Threshold 1 = no filter. in_soma: keep in_soma_ratio < thr. nc: keep nc_ratio == 0 or < thr."""
    out = granules.copy()
    out["nc_ratio"] = nc_ratio
    if in_soma_thr < 1:
        out = out[out["in_soma_ratio"] < in_soma_thr].copy()
    if nc_thr < 1:
        out = out[(out["nc_ratio"] == 0) | (out["nc_ratio"] < nc_thr)].copy()
    return out.reset_index(drop=True)

In [10]:
nc_ratio_all = compute_nc_ratio(granules, transcripts, nc_genes)

In [11]:
# 1. In-soma only (nc_thr=1); in_soma_thr in [0, 1e-4, 0.1, ..., 1]
in_soma_vals = np.concatenate([[0], [1e-4], np.arange(0.1, 1.05, 0.1)])


def _row(filtered, scenario, in_soma_thr, nc_thr):
    n = len(filtered)
    return {
        "scenario": scenario,
        "in_soma_thr": in_soma_thr,
        "nc_thr": nc_thr,
        "n_detections": n,
        "mean_sphere_r": filtered["sphere_r"].mean() if n else np.nan,
        "mean_in_soma_ratio": filtered["in_soma_ratio"].mean() if n else np.nan,
    }


rows_1 = [
    _row(apply_filters(granules, nc_ratio_all, in_soma_thr=t, nc_thr=1.0), "in_soma_only", t, 1.0)
    for t in in_soma_vals
]
df_1 = pd.DataFrame(rows_1)
df_1

,scenario,in_soma_thr,nc_thr,n_detections,mean_sphere_r,mean_in_soma_ratio
0,in_soma_only,0.0000,1.0,0,NaN,NaN
1,in_soma_only,0.0001,1.0,729894,1.010201,0.000000
2,in_soma_only,0.1000,1.0,736346,1.017808,0.000590
3,in_soma_only,0.2000,1.0,758604,1.027461,0.004820
4,in_soma_only,0.3000,1.0,796598,1.032681,0.016093
5,in_soma_only,0.4000,1.0,827077,1.034408,0.028028
6,in_soma_only,0.5000,1.0,848240,1.040275,0.037823
7,in_soma_only,0.6000,1.0,883196,1.045178,0.056881
8,in_soma_only,0.7000,1.0,928817,1.045879,0.085883
9,in_soma_only,0.8000,1.0,958713,1.049233,0.106471


In [12]:
# 2. NC only (in_soma_thr=1); nc_thr in [0, 0.1, ..., 1]
nc_vals = np.arange(0, 1.05, 0.1)
rows_2 = [
    _row(apply_filters(granules, nc_ratio_all, in_soma_thr=1.0, nc_thr=t), "nc_only", 1.0, t)
    for t in nc_vals
]
df_2 = pd.DataFrame(rows_2)
df_2

,scenario,in_soma_thr,nc_thr,n_detections,mean_sphere_r,mean_in_soma_ratio
0,nc_only,1.0,0.0,901070,0.970586,0.183764
1,nc_only,1.0,0.1,941486,1.008987,0.188551
2,nc_only,1.0,0.2,1022909,1.036183,0.205366
3,nc_only,1.0,0.3,1082379,1.039899,0.220711
4,nc_only,1.0,0.4,1103823,1.039620,0.227390
5,nc_only,1.0,0.5,1113009,1.040786,0.231116
6,nc_only,1.0,0.6,1127279,1.041092,0.237663
7,nc_only,1.0,0.7,1136987,1.039662,0.242566
8,nc_only,1.0,0.8,1141153,1.039728,0.244636
9,nc_only,1.0,0.9,1143264,1.040142,0.245695


In [13]:
# 3. Both filters; nc_thr=0.1 fixed, in_soma_thr in [0, 1e-4, 0.1, ..., 1]
rows_3 = [
    _row(apply_filters(granules, nc_ratio_all, in_soma_thr=t, nc_thr=0.1), "in_soma_and_nc", t, 0.1)
    for t in in_soma_vals
]
df_3 = pd.DataFrame(rows_3)
df_3

,scenario,in_soma_thr,nc_thr,n_detections,mean_sphere_r,mean_in_soma_ratio
0,in_soma_and_nc,0.0000,0.1,0,NaN,NaN
1,in_soma_and_nc,0.0001,0.1,672383,1.002231,0.000000
2,in_soma_and_nc,0.1000,0.1,677192,1.008333,0.000484
3,in_soma_and_nc,0.2000,0.1,692433,1.015479,0.003695
4,in_soma_and_nc,0.3000,0.1,719748,1.018404,0.012681
5,in_soma_and_nc,0.4000,0.1,742531,1.018070,0.022689
6,in_soma_and_nc,0.5000,0.1,756332,1.021378,0.029917
7,in_soma_and_nc,0.6000,0.1,779267,1.023295,0.044249
8,in_soma_and_nc,0.7000,0.1,809869,1.021173,0.067068
9,in_soma_and_nc,0.8000,0.1,828442,1.021719,0.082291


In [14]:
benchmark_results = pd.concat([df_1, df_2, df_3], ignore_index=True)
benchmark_results.to_csv(benchmark_path + "benchmark_filtering_results.csv", index=False)

In [15]:
sc.settings.verbosity = 0
genes = list(pd.read_csv(data_path + "processed_data/genes.csv").iloc[:, 0])
adata = sc.read_h5ad(data_path + "processed_data/adata.h5ad")
adata_neuron = adata[adata.obs["cell_type"].isin(["Glutamatergic", "GABAergic"])].copy()
gnl_genes = ["Camk2a", "Cplx2", "Slc17a7", "Ddn", "Syp", "Map1a", "Shank1", "Syn1", "Gria1", "Gria2", "Cyfip2", "Vamp2", "Bsn", "Slc32a1", "Nfasc", "Syt1", "Tubb3", "Nav1", "Shank3", "Mapt"]
mc = mcDETECT(type="MERSCOPE", transcripts=transcripts, gnl_genes=gnl_genes, nc_genes=None)

## Representative settings

Five settings: (1) no filter; (2) in-soma 0.1 only; (3) NC 0.1 only; (4) both 0.1; (5) in-soma 0.05, NC 0.1. Summary table; then per-setting profile (mcDETECT), merge with neurons, heatmap.

In [16]:
rep_settings = [(1.0, 1.0), (0.1, 1.0), (1.0, 0.1), (0.1, 0.1), (0.05, 0.1)]
rep_rows = []
for a, b in rep_settings:
    f = apply_filters(granules, nc_ratio_all, in_soma_thr=a, nc_thr=b)
    n = len(f)
    rep_rows.append({
        "setting": f"in_soma={a}, nc={b}",
        "in_soma_thr": a,
        "nc_thr": b,
        "n_detections": n,
        "mean_sphere_r": f["sphere_r"].mean() if n else np.nan,
        "mean_in_soma_ratio": f["in_soma_ratio"].mean() if n else np.nan,
    })
rep_summary = pd.DataFrame(rep_rows)
rep_summary

,setting,in_soma_thr,nc_thr,n_detections,mean_sphere_r,mean_in_soma_ratio
0,"in_soma=1.0, nc=1.0",1.00,1.0,1148130,1.040205,0.248143
1,"in_soma=0.1, nc=1.0",0.10,1.0,736346,1.017808,0.000590
2,"in_soma=1.0, nc=0.1",1.00,0.1,941486,1.008987,0.188551
3,"in_soma=0.1, nc=0.1",0.10,0.1,677192,1.008333,0.000484
4,"in_soma=0.05, nc=0.1",0.05,0.1,673355,1.004094,0.000052


In [17]:
gene_groups = {
    "Granule Markers": ["Camk2a", "Cplx2", "Slc17a7", "Syp", "Ddn", "Map1a", "Syn1", "Shank1", "Cyfip2", "Vamp2"],
    "Neuron Markers": ["Trpc4", "Gjc3", "Plekhb1", "Ntrk2", "Ntsr2", "Fn1", "Gnai2", "Shc3", "Reep3", "Chd9", "Acsbg1"],
}
target_genes = [g for group in gene_groups.values() for g in group]

In [18]:
heatmap_dir = benchmark_path + "benchmark_filtering_heatmaps/"
os.makedirs(heatmap_dir, exist_ok=True)
np.random.seed(42)

for i, (a, b) in enumerate(rep_settings):
    f = apply_filters(granules, nc_ratio_all, in_soma_thr=a, nc_thr=b)
    if len(f) == 0:
        print(f"Setting {i+1}: no granules, skip")
        continue
    ga = mc.profile(f, genes=genes)
    sc.pp.normalize_total(ga, target_sum=1e4)
    sc.pp.log1p(ga)
    ga = ga[np.random.permutation(ga.n_obs)].copy()
    adata_all = anndata.concat([ga, adata_neuron], axis=0, merge="same")
    adata_all.var["genes"] = adata_all.var.index
    adata_all.obs["type"] = pd.Categorical(["Granules"] * ga.n_obs + ["Neurons"] * adata_neuron.n_obs, categories=["Granules", "Neurons"], ordered=True)
    sc.set_figure_params(scanpy=True, fontsize=10)
    sc.pl.heatmap(adata_all, target_genes, groupby="type", log=True, cmap="Reds", standard_scale="var", swap_axes=True, figsize=(8, 4), show=False)
    plt.savefig(heatmap_dir + f"heatmap_in{a}_nc{b}.jpeg", dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Setting {i+1} (in_soma={a}, nc={b}): n={len(f)}, saved.")

Setting 1 (in_soma=1.0, nc=1.0): n=1148130, saved.
Setting 2 (in_soma=0.1, nc=1.0): n=736346, saved.
Setting 3 (in_soma=1.0, nc=0.1): n=941486, saved.
Setting 4 (in_soma=0.1, nc=0.1): n=677192, saved.
Setting 5 (in_soma=0.05, nc=0.1): n=673355, saved.
